In [81]:
import numpy as np
import tensorflow
from tensorflow.keras.models import Sequential
from tensorflow.keras import layers
from tensorflow.keras.layers import Dense,Flatten, Dropout
from tensorflow.keras.layers import Conv2D,MaxPooling2D
from keras.preprocessing.image import ImageDataGenerator

In [82]:
train_datagen = ImageDataGenerator(rescale=1./255, shear_range=0.2, zoom_range=0.2, horizontal_flip=True)
test_datagen = ImageDataGenerator(rescale=1./255)

In [83]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [84]:
x_train = train_datagen.flow_from_directory(r'/content/drive/MyDrive/img/train', target_size=(64, 64), batch_size=3, color_mode='grayscale', class_mode='categorical')
x_test = test_datagen.flow_from_directory(r'/content/drive/MyDrive/img/test', target_size=(64, 64), batch_size=3, color_mode='grayscale', class_mode='categorical')

Found 594 images belonging to 6 classes.
Found 30 images belonging to 6 classes.


In [85]:
print(x_train.class_indices)

{'0': 0, '1': 1, '2': 2, '3': 3, '4': 4, '5': 5}


In [86]:
model=Sequential()

In [87]:
model.add(Conv2D(32, (3, 3), input_shape=(64, 64, 1), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(32, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Flatten())

In [88]:
model.add(Dense(units=512, activation='relu'))
model.add(Dense(units=6, activation='softmax'))
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_2 (Conv2D)           (None, 62, 62, 32)        320       
                                                                 
 max_pooling2d_2 (MaxPooling  (None, 31, 31, 32)       0         
 2D)                                                             
                                                                 
 conv2d_3 (Conv2D)           (None, 29, 29, 32)        9248      
                                                                 
 max_pooling2d_3 (MaxPooling  (None, 14, 14, 32)       0         
 2D)                                                             
                                                                 
 flatten_1 (Flatten)         (None, 6272)              0         
                                                                 
 dense_2 (Dense)             (None, 512)              

In [89]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [90]:
model.fit_generator(
    generator=x_train, steps_per_epoch=len(x_train),
    epochs=20, validation_data=x_test, validation_steps=len(x_test)
)

Epoch 1/20


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  This is separate from the ipykernel package so we can avoid doing imports until


198/198 [==============================] - 14s 69ms/step - loss: 1.3176 - accuracy: 0.4933 - val_loss: 0.6846 - val_accuracy: 0.6667
Epoch 2/20
198/198 [==============================] - 13s 68ms/step - loss: 0.5931 - accuracy: 0.7727 - val_loss: 0.5165 - val_accuracy: 0.8000
Epoch 3/20
198/198 [==============================] - 14s 71ms/step - loss: 0.3975 - accuracy: 0.8502 - val_loss: 0.4175 - val_accuracy: 0.8333
Epoch 4/20
198/198 [==============================] - 13s 68ms/step - loss: 0.2846 - accuracy: 0.8939 - val_loss: 0.2699 - val_accuracy: 0.9000
Epoch 5/20
198/198 [==============================] - 13s 68ms/step - loss: 0.2058 - accuracy: 0.9242 - val_loss: 0.2757 - val_accuracy: 0.9333
Epoch 6/20
198/198 [==============================] - 13s 67ms/step - loss: 0.1634 - accuracy: 0.9310 - val_loss: 0.3208 - val_accuracy: 0.8667
Epoch 7/20
198/198 [==============================] - 14s 69ms/step - loss: 0.0901 - accuracy: 0.9764 - val_loss: 0.3185 - val_accuracy: 0.8333
Epo

In [91]:
model.save('ges.pkl')
model_json = model.to_json()
with open("model-bw.json", "w") as json_file:
     json_file.write(model_json)

In [92]:
import numpy as np
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing import image
model = load_model("ges.pkl")
img = image.load_img(r'/content/drive/MyDrive/img/test/0/2.jpg', grayscale=True, target_size=(64,64))
x = image.img_to_array(img)
x.shape

/usr/local/lib/python3.7/dist-packages/keras/utils/image_utils.py:382: UserWarning: grayscale is deprecated. Please use color_mode = "grayscale"
  warnings.warn('grayscale is deprecated. Please use '


(64, 64, 1)

In [93]:
x = np.expand_dims(x,axis=0)
x.shape

(1, 64, 64, 1)

In [94]:
pred_x = model.predict(x)
pred_x=np.argmax(pred_x,axis=1)
pred_x

1/1 [==============================] - 0s 73ms/step


array([0])

In [96]:
index=['0', '1', '2', '3', '4', '5']
result=str(index[pred_x[0]])
result

'0'

In [95]:
import numpy as np
p=[]
for i in range(0,6):
  for j in range(0,5):
    img = image.load_img(r"/content/drive/MyDrive/img/test/"+str(i)+"/"+str(j)+".jpg", grayscale=True, target_size=(64,64))
    x = image.img_to_array(img)
    x = np.expand_dims(x,axis=0)
    pred = np.argmax(model.predict(x),axis=-1)
    p.append(pred)
print(p)

1/1 [==============================] - 0s 25ms/step
[array([0]), array([0]), array([0]), array([0]), array([0]), array([1]), array([1]), array([1]), array([1]), array([1]), array([2]), array([2]), array([1]), array([2]), array([2]), array([3]), array([3]), array([3]), array([2]), array([3]), array([4]), array([4]), array([4]), array([4]), array([4]), array([5]), array([5]), array([5]), array([5]), array([5])]


In [97]:
result = []
index=['0','1','2','3','4','5']
for i in p:
  result.append(index[i[0]])
print(result)

['0', '0', '0', '0', '0', '1', '1', '1', '1', '1', '2', '2', '1', '2', '2', '3', '3', '3', '2', '3', '4', '4', '4', '4', '4', '5', '5', '5', '5', '5']


In [98]:
from flask import Flask,render_template,request

In [99]:
%mkdir templates -p

In [114]:
%%writefile templates/home.html



<html>
  <head>
    <meta charset="utf-8">
    <meta name="viewport" content="width=device-width, initial-scale=1">
    <link href="https://cdn.jsdelivr.net/npm/bootstrap@5.2.2/dist/css/bootstrap.min.css" rel="stylesheet" integrity="sha384-Zenh87qX5JnK2Jl0vWa8Ck2rdkQ2Bzep5IDxbcnCeuOxjzrPF/et3URy9Bv1WTRi" crossorigin="anonymous">
    <!--link rel="stylesheet" href="home.css"-->
    <style>
      body {
        background-image: url('https://tse2.mm.bing.net/th?id=OIP.0jyFTQx2umY6fRIB-q3FqAHaHO&pid=Api&P=0');
        background-repeat: no-repeat;
        background-attachment: fixed;
        background-size: cover;
      }
      </style>
    <title>
        Gesture Recognition System
    </title>
</head>


<body>
<!--navigation bar-->
<nav class="navbar navbar-expand-md navbar-light">
  <div class="container-xxl">
    <a href="{{ url_for('home')}}" class="navbar-brand">
      <span class="fw-normal  text-black">
        Gesture System
      </span>
    </a>
    <!--menu(toggle) button for mobile -->
    <button class="navbar-toggler" type="button" data-bs-toggle="collapse" data-bs-target="#main-nav" aria-controls="main-nav" aria-expanded="false" aria-label="Toggle navigation">
      <span class="navbar-toggler-icon"></span>
    </button>
    <!--navbar links-->
    <div class="collapse navbar-collapse justify-content-end align-center" id="main-nav">
      <ul class="navbar-nav">
        <li class="nav-item">
          <a href="{{ url_for('home')}}" class="nav-link active">Home</a>
        </li>
        <li class="nav-item">
          <a href="{{ url_for('intro')}}" class="nav-link active">Introduction</a>
        </li>
        <li class="nav-item">
          <a href="{{ url_for('image1')}}" class="nav-link active">Launch</a>
        </li>
      </ul>
    </div>
  </div>
</nav>

  <div class="container-lg m-3">
    <div class="row justify-content-center align-items-center">
      <div class="text-center text-md-center">
        <h1>
          <div class="display-2 text-dark">Hand Gesture Recognition</div>
          <div class="display-5 text-dark-50">For Sterile Browsing of Radiology Images</div>
        </h1>
        <div class="text-center d-none d-md-block m-1">
          <img class="rounded mx-auto d-block" src="https://cdn1.iconfinder.com/data/icons/hand-gestures-66/48/hand-gesture-count-two-512.png" alt="gesture img">
        </div>
        <div class=" my-2 text-center">
          <a type="button" class="btn btn-primary btn-lg" href="{{ url_for('intro')}}">Click Here To Know More</a>  
          <a type="button" class="btn btn-success btn-lg" href="{{ url_for('image1')}}">Launch</a>   
          </button>
      </div>
    </div>

    

    <script src="https://cdn.jsdelivr.net/npm/bootstrap@5.2.2/dist/js/bootstrap.bundle.min.js" integrity="sha384-OERcA2EqjJCMA+/3y+gxIOqMEjwtxJY7qPCqsdltbNJuaOe923+mo//f6V8Qbsw3" crossorigin="anonymous"></script>
</body>

</html>

Overwriting templates/home.html


In [101]:
%%writefile templates/intro.html
<html>
  <head>
    <meta charset="utf-8">
    <meta name="viewport" content="width=device-width, initial-scale=1">
    <link href="https://cdn.jsdelivr.net/npm/bootstrap@5.2.2/dist/css/bootstrap.min.css" rel="stylesheet" integrity="sha384-Zenh87qX5JnK2Jl0vWa8Ck2rdkQ2Bzep5IDxbcnCeuOxjzrPF/et3URy9Bv1WTRi" crossorigin="anonymous">
    <!--link rel="stylesheet" href="home.css"-->
    <style>
      body {
        background-image: url('https://tse2.mm.bing.net/th?id=OIP.0jyFTQx2umY6fRIB-q3FqAHaHO&pid=Api&P=0');
        background-repeat: no-repeat;
        background-attachment: fixed;
        background-size: cover;
      }
      </style>
    <title>
        Gesture Recognition 
    </title>
</head>


<body>
<!--navigation bar-->
<nav class="navbar navbar-expand-md navbar-light">
  <div class="container-xxl">
    <a href="{{ url_for('home')}}" class="navbar-brand">
      <span class="fw-normal  text-black">
        Gesture System
      </span>
    </a>
    <!--menu(toggle) button for mobile -->
    <button class="navbar-toggler" type="button" data-bs-toggle="collapse" data-bs-target="#main-nav" aria-controls="main-nav" aria-expanded="false" aria-label="Toggle navigation">
      <span class="navbar-toggler-icon"></span>
    </button>
    <!--navbar links-->
    <div class="collapse navbar-collapse justify-content-end align-center" id="main-nav">
      <ul class="navbar-nav">
        <li class="nav-item">
          <a href="{{ url_for('home')}}" class="nav-link active">Home</a>
        </li>
        <li class="nav-item">
          <a href="{{ url_for('intro')}}" class="nav-link active">Introduction</a>
        </li>
        <li class="nav-item">
          <a href="{{ url_for('image1')}}" class="nav-link active">Launch</a>
        </li>
      </ul>
    </div>
  </div>
</nav>

  <div class="container-lg m-3">
    <div class="row justify-content-center align-items-center">
      <div class="text-center text-md-center">
        <h1>
          <div class="display-2 text-dark">Hand Gesture Recognition</div>
          <div class="display-4 text-dark">About the System</div>
          <br>
          <small class="lead text-dark">
           <h2>Hand gestures are a form of nonverbal communication that can be used in several fields such as communication between deaf-mute people, robot control, human–computer interaction (HCI), home automation and medical applications. Research papers based on hand gestures have adopted many different techniques, including those based on instrumented sensor technology and computer vision
            Then, we will be predicting the labels based on the CNN trained model weights of hand gestures using that predicted labels we apply if conditions  to control some of the actions like reshaping , blur, flip of the given image.
          </h2> </small>
        </h1>
        <div class="text-center d-none d-md-block m-1">
          <img class="rounded mx-auto d-block" src="https://media.tenor.com/GwMnZWwxip0AAAAC/hand-finger-counting.gif" alt="gesture img">
        </div>
        <div class=" my-2 text-center">
          <a type="button" class="btn btn-primary btn-lg" href="{{ url_for('intro')}}">Click Here To Know More</a>  
          <a type="button" class="btn btn-success btn-lg" href="{{ url_for('image1')}}">Launch</a>   
          </button>
      </div>
    </div>

    

    <script src="https://cdn.jsdelivr.net/npm/bootstrap@5.2.2/dist/js/bootstrap.bundle.min.js" integrity="sha384-OERcA2EqjJCMA+/3y+gxIOqMEjwtxJY7qPCqsdltbNJuaOe923+mo//f6V8Qbsw3" crossorigin="anonymous"></script>
</body>

</html>

Overwriting templates/intro.html


In [102]:
%%writefile templates/launch.html
<html lang="en">

  <head>
  <meta charset="utf-8">
      <meta http-equiv="X-UA-Compatible" content="IE=edge">
      <meta name="viewport" content="width=device-width, initial-scale=0.6">
      <script src="https://ajax.googleapis.com/ajax/libs/jquery/3.3.1/jquery.min.js"></script>
      <link href="https://fonts.googleapis.com/icon?family=Material+Icons" rel="stylesheet">
      <meta charset="UTF-8">
      <title>Predict</title>
      <link href="https://cdn.bootcss.com/bootstrap/4.0.0/css/bootstrap.min.css" rel="stylesheet">
   
      
      <link href="{{ url_for('static', filename='css/main.css') }}" rel="stylesheet">           
      <style>
        body {
          background-image: url('https://media.npr.org/assets/img/2018/08/16/gettyimages-507850475_wide-e729499092c4106cff752cea8a2c49623c1595d3.jpg?s=1400');
          background-repeat: no-repeat;
          background-attachment: fixed;
          background-size: cover;
        }
        </style>
  </head>
  
  <body>
  <!--navigation bar-->
    <nav class="navbar navbar-expand-md navbar-light">
  <div class="container-xxl">
    <a href="{{ url_for('home')}}" class="navbar-brand">
      <span class="fw-normal  text-black">
        Gesture Recognition System
      </span>
    </a>
    <!--menu(toggle) button for mobile -->
    <button class="navbar-toggler" type="button" data-bs-toggle="collapse" data-bs-target="#main-nav" aria-controls="main-nav" aria-expanded="false" aria-label="Toggle navigation">
      <span class="navbar-toggler-icon"></span>
    </button>
    <!--navbar links-->
    <div class="collapse navbar-collapse justify-content-end align-center" id="main-nav">
      <ul class="navbar-nav">
        <li class="nav-item">
          <a href="{{ url_for('home')}}" class="nav-link active">Home</a>
        </li>
        <li class="nav-item">
          <a href="{{ url_for('intro')}}" class="nav-link active">Introduction</a>
        </li>
        <li class="nav-item">
          <a href="{{ url_for('image1')}}" class="nav-link active">Launch</a>
        </li>
      </ul>
    </div>
  </div>
    </nav>
    <br>
    
    <div1 style="text-align: center;"><h1><font color="Black" size="6" font-family="Roboto">Hand Gesture Recognition</h1><br>
    <p><i><font color="Black" size="4" font-family="sans-serif"></i>Provide an image for which you want to perform various operations</p>
    <br>
          <div>
            <h4>Upload Image Here</h4>
        <form action = "http://localhost:5000/" id="upload-file" method="post" enctype="multipart/form-data">
          <label for="imageUpload" class="upload-label">
            Choose...
          </label>
          <input type="file" name="image" id="imageUpload" accept=".png, .jpg, .jpeg,.pdf">
        </form>
              <center>
        <div class="image-section" style="display:none;">
          <div class="img-preview">
            <div id="imagePreview">
            </div>
          </div>
          <div>
            <button type="button" class="btn btn-info btn-lg " id="btn-predict">Predict!</button>
          </div>
        </div>
        <div class="loader" style="display:none;"></div>
              </center>
      </div>
      </div1>
         
          <footer>
      <script src="{{ url_for('static', filename='js/main.js') }}" type="text/javascript"></script>    
  </footer>
          
  </html>    
  

Overwriting templates/launch.html


In [68]:
!pip3 install SimpleITK

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [103]:
!pip install flask-ngrok

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [116]:
from flask_ngrok import run_with_ngrok

In [117]:
from __future__ import print_function

import SimpleITK as sitk
import sys
import os

In [118]:
import operator
import cv2 # opencv library
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import numpy as np

In [119]:
from tensorflow.keras.models import load_model#to load our trained model
import os
from werkzeug.utils import secure_filename

In [120]:
pip install pyngrok

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [121]:
from pyngrok import ngrok

In [122]:
app = Flask(__name__,template_folder="templates")
app = Flask('  ')
run_with_ngrok(app)

In [123]:
port=5000
ngrok.set_auth_token('2HXrgO3lSMBp30J01GCjcFXlMaZ_6BToXQgLussvaeKTkKMFu')

In [124]:
from flask import Flask,render_template,request
# Flask-It is our framework which we are going to use to run/serve our application.
#request-for accessing file which was uploaded by the user on our application.
import operator
import cv2 # opencv library
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import numpy as np
from tensorflow.keras.models import load_model#to load our trained model
import os
from werkzeug.utils import secure_filename




In [125]:
model=load_model('ges.pkl')
print("Loaded model from disk")

Loaded model from disk


In [ ]:
@app.route('/')# route to display the home page
def home():
    return render_template('home.html')#rendering the home page


@app.route('/intro') # routes to the intro page
def intro():
    return render_template('intro.html')#rendering the intro page

@app.route('/image1',methods=['GET','POST'])# routes to the index html
def image1():
    return render_template("launch.html")


@app.route('/predict',methods=['GET', 'POST'])# route to show the predictions in a web UI
def launch():
    if request.method == 'POST':
        print("inside image")
        f = request.files['image']
        basepath = os.path.dirname(__file__)
        file_path = os.path.join(basepath, 'uploads', secure_filename(f.filename))
        f.save(file_path)   
        print(file_path)
        cap = cv2.VideoCapture(0)
        while True:
            _, frame = cap.read() #capturing the video frame values
            # Simulating mirror image
            frame = cv2.flip(frame, 1) 
            # Got this from collect-data.py
            # Coordinates of the ROI
            x1 = int(0.5*frame.shape[1]) 
            y1 = 10
            x2 = frame.shape[1]-10
            y2 = int(0.5*frame.shape[1])
            # Drawing the ROI
            # The increment/decrement by 1 is to compensate for the bounding box
            cv2.rectangle(frame, (x1-1, y1-1), (x2+1, y2+1), (255,0,0) ,1)
            # Extracting the ROI
            roi = frame[y1:y2, x1:x2]
            
            # Resizing the ROI so it can be fed to the model for prediction
            roi = cv2.resize(roi, (64, 64)) 
            roi = cv2.cvtColor(roi, cv2.COLOR_BGR2GRAY)
            _, test_image = cv2.threshold(roi, 120, 255, cv2.THRESH_BINARY)
            cv2.imshow("test", test_image)
            # Batch of 1
            result = model.predict(test_image.reshape(1, 64, 64, 1))
            prediction = {'ZERO': result[0][0], 
                          'ONE': result[0][1], 
                          'TWO': result[0][2],
                          'THREE': result[0][3],
                          'FOUR': result[0][4],
                          'FIVE': result[0][5]}
            # Sorting based on top prediction
            prediction = sorted(prediction.items(), key=operator.itemgetter(1), reverse=True)
            
            # Displaying the predictions
            cv2.putText(frame, prediction[0][0], (10, 120), cv2.FONT_HERSHEY_PLAIN, 1, (0,255,255), 1)    
            cv2.imshow("Frame", frame)
            
            #loading an image
            image1=cv2.imread(file_path)
            if prediction[0][0]=='ONE':
                resized = cv2.resize(image1, (200, 200))
                cv2.imshow("Fixed Resizing", resized)
                key=cv2.waitKey(3000)
                if (key & 0xFF) == ord("1"):
                    cv2.destroyWindow("Fixed Resizing")
            
            elif prediction[0][0]=='ZERO':
                cv2.rectangle(image1, (480, 170), (650, 420), (0, 0, 255), 2)
                cv2.imshow("Rectangle", image1)
                cv2.waitKey(0)
                key=cv2.waitKey(3000)
                if (key & 0xFF) == ord("0"):
                    cv2.destroyWindow("Rectangle")
                
            elif prediction[0][0]=='TWO':
                (h, w, d) = image1.shape
                center = (w // 2, h // 2)
                M = cv2.getRotationMatrix2D(center, -45, 1.0)
                rotated = cv2.warpAffine(image1, M, (w, h))
                cv2.imshow("OpenCV Rotation", rotated)
                key=cv2.waitKey(3000)
                if (key & 0xFF) == ord("2"):
                    cv2.destroyWindow("OpenCV Rotation")
                
            elif prediction[0][0]=='THREE':
                blurred = cv2.GaussianBlur(image1, (21, 21), 0)
                cv2.imshow("Blurred", blurred)
                key=cv2.waitKey(3000)
                if (key & 0xFF) == ord("3"):
                    cv2.destroyWindow("Blurred")

            elif prediction[0][0]=='FOUR':
                resized = cv2.resize(image1, (400, 400))
                cv2.imshow("Fixed Resizing", resized)
                key=cv2.waitKey(3000)
                if (key & 0xFF) == ord("4"):
                    cv2.destroyWindow("Fixed Resizing")

            elif prediction[0][0]=='FIVE':
                '''(h, w, d) = image1.shape
                center = (w // 2, h // 2)
                M = cv2.getRotationMatrix2D(center, 45, 1.0)
                rotated = cv2.warpAffine(image1, M, (w, h))'''
                gray = cv2.cvtColor(image1, cv2.COLOR_RGB2GRAY)
                cv2.imshow("OpenCV Gray Scale", gray)
                key=cv2.waitKey(3000)
                if (key & 0xFF) == ord("5"):
                    cv2.destroyWindow("OpenCV Gray Scale")

            else:
                continue
            
            
            interrupt = cv2.waitKey(10)
            if interrupt & 0xFF == 27: # esc key
                break
                
         
        cap.release()
        cv2.destroyAllWindows()
    return render_template("home.html")
     
if __name__ == "__main__":
   # running the app
    app.run()

 * Serving Flask app "  " (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


INFO:werkzeug: * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)


 * Running on http://323d-35-237-245-185.ngrok.io
 * Traffic stats available on http://127.0.0.1:4040


INFO:werkzeug:127.0.0.1 - - [14/Nov/2022 17:28:08] "GET / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [14/Nov/2022 17:28:09] "GET /favicon.ico HTTP/1.1" 404 -
INFO:werkzeug:127.0.0.1 - - [14/Nov/2022 17:28:16] "GET /intro HTTP/1.1" 200 -
